In [1]:
import tensorflow as tf
import numpy as np

class SequentialTFModel:
    def __init__(self, input_dim):
        self.layers = []
        self.input_dim = input_dim
        self.prev_dim = input_dim

    def add(self, units, activation='relu'):
        W = tf.Variable(tf.random.normal([self.prev_dim, units], stddev=0.1), trainable=True)
        b = tf.Variable(tf.zeros([units]), trainable=True)
        self.layers.append({'W': W, 'b': b, 'activation': activation})
        self.prev_dim = units

    def forward(self, X):
        out = X
        for layer in self.layers:
            Z = tf.matmul(out, layer['W']) + layer['b']
            if layer['activation'] == 'relu':
                out = tf.nn.relu(Z)
            elif layer['activation'] == 'sigmoid':
                out = tf.nn.sigmoid(Z)
            else:
                raise ValueError("Unsupported activation")
        return out

    def train(self, X, Y, epochs=100, lr=0.01, loss_fn='mse'):
        optimizer = tf.optimizers.Adam(lr)

        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                predictions = self.forward(X)
                if loss_fn == 'mse':
                    loss = tf.reduce_mean((predictions - Y) ** 2)
                elif loss_fn == 'bce':
                    loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(Y, predictions))
                else:
                    raise ValueError("Unsupported loss")

            variables = []
            for layer in self.layers:
                variables += [layer['W'], layer['b']]

            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

    def predict(self, X):
        return self.forward(X)

In [3]:
# forward function
def autoencoder_forward(X):
    encoded = encoder.forward(X)
    decoded = decoder.forward(encoded)
    return decoded

# trainable variables
def get_autoencoder_variables():
    vars = []
    for model in [encoder, decoder]:
        for layer in model.layers:
            vars += [layer['W'], layer['b']]
    return vars

# Train function
def train_autoencoder(X, epochs=100, lr=0.01):
    optimizer = tf.optimizers.Adam(lr)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            reconstructed = autoencoder_forward(X)
            loss = tf.reduce_mean((reconstructed - X) ** 2)

        vars = get_autoencoder_variables()
        grads = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(grads, vars))

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

In [15]:
# Dummy data
input_dim = 16
np.random.seed(42)
X_data = np.random.rand(1000, input_dim).astype(np.float32)

# encoder
encoder = SequentialTFModel(input_dim)
encoder.add(12, activation='relu')
encoder.add(8, activation='relu')
encoder.add(4, activation='relu') 

# decoder 
decoder = SequentialTFModel(4)
decoder.add(8, activation='relu')
decoder.add(12, activation='relu')
decoder.add(input_dim, activation='sigmoid')  

X_tensor = tf.convert_to_tensor(X_data)

# Train autoencoder
train_autoencoder(X_tensor, epochs=100, lr=0.01)

Epoch 0, Loss: 0.0834
Epoch 10, Loss: 0.0833
Epoch 20, Loss: 0.0833
Epoch 30, Loss: 0.0824
Epoch 40, Loss: 0.0787
Epoch 50, Loss: 0.0778
Epoch 60, Loss: 0.0773
Epoch 70, Loss: 0.0765
Epoch 80, Loss: 0.0752
Epoch 90, Loss: 0.0738


In [16]:
# Predict on some inputs
reconstructed = autoencoder_forward(X_tensor[:5])
print("Original:\n", X_tensor[:5].numpy())
print("Reconstructed:\n", reconstructed.numpy())

Original:
 [[0.37454012 0.9507143  0.7319939  0.5986585  0.15601864 0.15599452
  0.05808361 0.8661761  0.601115   0.7080726  0.02058449 0.96990985
  0.83244264 0.21233912 0.18182497 0.1834045 ]
 [0.30424225 0.52475643 0.43194503 0.29122913 0.6118529  0.13949387
  0.29214466 0.36636186 0.45606998 0.785176   0.19967379 0.5142344
  0.59241456 0.04645041 0.60754484 0.17052412]
 [0.06505159 0.94888556 0.965632   0.80839735 0.30461377 0.09767211
  0.684233   0.4401525  0.12203824 0.4951769  0.03438852 0.9093204
  0.25877997 0.66252226 0.31171107 0.52006805]
 [0.54671025 0.18485446 0.96958464 0.77513283 0.93949896 0.89482737
  0.5979     0.9218742  0.08849251 0.19598286 0.04522729 0.32533032
  0.3886773  0.27134904 0.8287375  0.35675332]
 [0.2809345  0.54269606 0.14092423 0.802197   0.07455064 0.9868869
  0.77224475 0.19871569 0.00552212 0.81546146 0.7068573  0.7290072
  0.77127033 0.07404465 0.35846573 0.11586906]]
Reconstructed:
 [[0.51007825 0.5003284  0.4438317  0.46769512 0.52557915 0.54